In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from scipy import stats

In [21]:
# Load data
TFT_predictions = np.load('./Tensors/TFT_predictions.npy', allow_pickle=True)
naive_predictions = np.load('./Tensors/baseline_predictions (1).npy', allow_pickle=True)
tests = np.load('./Tensors/actual_sales.npy', allow_pickle=True)
data = pd.read_csv('data_processed_plant_item_encoded_13_03_22.csv')

In [22]:
#classifications = np.load('forecast_classification.npy')

In [23]:
def create_ensemble_preds(clf, TFT_preds_clf):
    TFT_ensemble_predictions = np.zeros((len(clf), 6))
    for i in range(len(clf)):
        if clf[i] == 0:
            TFT_ensemble_predictions[i, :] = np.zeros((6))
        else:
            TFT_ensemble_predictions[i, :] = TFT_preds_clf[i, :]
    return TFT_ensemble_predictions
#TFT_ensemble_predictions =create_ensemble_preds(classifications, TFT_predictions)

In [48]:
def WMAPE_calc(preds, tests):
    flat_pred = np.array(preds).flatten()
    flat_test = np.array(tests).flatten()
    residuals = []
    for i in range(len(flat_pred)):
        residuals.append(abs(flat_test[i] - flat_pred[i]))
    WMAPE = 100 * (np.sum(residuals) / np.sum(flat_test))
    return WMAPE

def WMAPE2_calc(preds, tests):
    preds = preds
    tests = tests
    total_vol = np.sum(tests)
    residuals = 0
    total_vol = np.sum(tests)
    for i in range(preds.shape[0]):
        pred = preds[i, :]
        for t in range(len(pred)):
            residuals += abs(pred[t] - tests[i, t])
    print(f' total residual : {residuals}')
    print(f' total volume: {total_vol}')
    WMAPE_2 = (residuals / total_vol) * 100
    return WMAPE_2

def metric_test(preds, tests):
    MSEs = []
    MAPEs = []
    MAE = []
    for i in range(preds.shape[0]):
        pred = preds[i, :]
        test = tests[i, :]
        MSEs.append(mean_squared_error(test, pred))
        MAPEs.append(mean_absolute_percentage_error(test + 1, pred + 1))
        MAE.append(mean_absolute_error(test, pred))
    WMAPE = WMAPE_calc(preds, tests)
    WMAPE_2 = WMAPE2_calc(preds, tests)
    #WMAPE_2 = WMAPE_calc_2(preds, tests)
    print('mean squared error:                              ', round(np.mean(MSEs), 2))
    print('mean absolute error:                             ', round(np.mean(MAE), 2))
    print('mean absolute percentage error:          ', round(np.mean(MAPEs), 2), '%')
    #print('mean absolute percentage error weighted: ', round(WMAPE, 2), '%')
    print('mean absolute percentage error weighted: ', round(WMAPE_2, 2), '%')

def compare_performance(model_dict, tests):
    for model in list(model_dict.keys()):
        print(model)
        print('')
        metric_test(model_dict[model], tests)
        print('')


model_dict = {'TFT':TFT_predictions, 'naive':naive_predictions}
compare_performance(model_dict, tests)

TFT

 total residual : 3017482.9082693593
 total volume: 3017473.0
mean squared error:                               70773.31
mean absolute error:                              48.12
mean absolute percentage error:           0.39 %
mean absolute percentage error weighted:  100.0 %

naive

 total residual : 2543879.0
 total volume: 3017473.0
mean squared error:                               41915.38
mean absolute error:                              40.57
mean absolute percentage error:           3.35 %
mean absolute percentage error weighted:  84.3 %



In [49]:
print(tests[0:4])

[[203.   0.   0.   0.   0.   0.]
 [  0.   0.  10.   3.   0.   0.]
 [  0.   0.   8.   8.   4.   1.]
 [  0.  40.  38.   0.  78.  42.]]


In [44]:
def sum_over_plants(array, data):
    timeseries_list = list(data['timeseries'].unique())
    item_dict = {}
    for i in range(len(timeseries_list)):
        timeseries = data[data['timeseries'] == timeseries_list[i]]
        itemID = list(timeseries['ParentItemID'].unique())[0]
        if itemID not in list(item_dict.keys()):
            item_dict[itemID] = array[i, :]
        else:
            prev_vol = item_dict[itemID]
            #print(prev_vol)
            #print(array[i, :])
            new_vol = prev_vol + array[i, :]
            #print(new_vol)
            #print('')
            item_dict[itemID] = new_vol
    print(len(list(item_dict.keys())))
    print(len(data['ParentItemID'].unique()))
    print('')
    return item_dict

In [45]:
test_item_dict = sum_over_plants(tests, data)

TFT_preds_dict = sum_over_plants(TFT_predictions, data)

naive_preds_dict = sum_over_plants(naive_predictions, data)

1068
1068

1068
1068

1068
1068



In [46]:
def WMAPE2_calc(pred_dict, test_dict, model):
    residuals = 0
    total_vol = np.sum(list(test_dict.values()))
    for itemID in list(pred_dict.keys()):
        pred = pred_dict[itemID] 
        test = test_dict[itemID] 
        for t in range(len(pred)):
            residuals += abs(pred[t] - test[t])
    
    print(f'total residual : {residuals} for {model}')
    print(f'total volume: {total_vol} for {model}')
    WMAPE_2 = (residuals / total_vol) * 100
    print('Weighted and aggregated MAPE for:', model, ':', round(WMAPE_2, 2), '%')

In [47]:
models = ['TFT', 'Naive']

WMAPE2_calc(TFT_preds_dict, test_item_dict, models[0])

WMAPE2_calc(naive_preds_dict, test_item_dict, models[1])

total residual : 3017391.1049483516 for TFT
total volume: 3017473.0 for TFT
Weighted and aggregated MAPE for: TFT : 100.0 %
total residual : 1882003.0 for Naive
total volume: 3017473.0 for Naive
Weighted and aggregated MAPE for: Naive : 62.37 %
